# LMSYS - Chatbot Arena Human Preference Predictions - Embeddings using TPU

In [1]:
!pip install ipywidgets --upgrade


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import re
import string
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, TFBertModel
from tqdm.notebook import tqdm

E0000 00:00:1721039404.148863    2893 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0715 10:30:04.157117780    2893 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0715 10:30:04.157133348    2893 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0715 10:30:04.157136642    2893 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0715 10:30:04.157139105    2893 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0715 10:30:04.157141545    2893 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [4]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Config TPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All TPU devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("ERROR: TPU not found, defaulting to CPU/GPU strategy")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1721039413.791086    2893 service.cc:145] XLA service 0x5c0dcab38a90 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721039413.791150    2893 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1721039413.791155    2893 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1721039413.791158    2893 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1721039413.791161    2893 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1721039413.791164    2893 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1721039413.791166    2893 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1721039413.791169    2893 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1721039413.791171    2893 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [6]:
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

In [7]:
# Cleaning data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [8]:
# Cleaning texts
train_data['prompt_clean'] = train_data['prompt'].apply(clean_text)
train_data['response_a_clean'] = train_data['response_a'].apply(clean_text)
train_data['response_b_clean'] = train_data['response_b'].apply(clean_text)

In [9]:
# Inicialize BERT
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
@tf.function
def get_bert_embeddings(texts):
    inputs = bert_tokenizer(texts, 
                            return_tensors='tf', 
                            padding=True, 
                            truncation=True, 
                            max_length=512)
    outputs = bert_model(inputs)
    return outputs.last_hidden_state[:, 0, :]

In [11]:
def process_column(column_data):
    column_data = column_data.dropna().tolist()
    column_data = [str(text) for text in column_data]  
    dataset = tf.data.Dataset.from_tensor_slices(column_data)
    dataset = dataset.batch(8)  
    
    embeddings = []
    for batch in tqdm(dataset, desc="Processing batches"):
        batch_list = [str(text) for text in batch.numpy().tolist()]  
        batch_embeddings = get_bert_embeddings(batch_list)
        embeddings.append(batch_embeddings)
    
    return np.concatenate(embeddings, axis=0)

In [12]:
def add_embeddings_to_dataframe(df, column_names):
    for column in tqdm(column_names, desc="Processing columns"):
        print(f"Processing column: {column}")
        embeddings = process_column(df[column])
        df[f'{column}_embedding'] = list(embeddings)
    return df

In [13]:
columns_to_embed = ['prompt_clean', 'response_a_clean', 'response_b_clean']

In [ ]:
with strategy.scope():
    train_data = add_embeddings_to_dataframe(train_data, columns_to_embed)

Processing columns:   0%|          | 0/3 [00:00<?, ?it/s]

Processing column: prompt_clean


Processing batches:   0%|          | 0/7185 [00:00<?, ?it/s]

In [ ]:
train_data.to_csv('/kaggle/working/train_data_embeddings.csv', index=False)